In [11]:
import ast
import networkx as nx

In [12]:
with open('main.py') as f:
    source = ''.join(f.readlines())

In [13]:
main_ast = ast.parse(source)

In [24]:
class ASTBuilder:
    def __init__(self, root):
        self.G = nx.Graph()

        self.root = self.AntiSingleton(root)
        self.build()

    class AntiSingleton:
        def __init__(self, singleton):
            self.value = singleton

    def build(self):
        self.add_node(self.root)
        self.visit(self.root)

    def go_further(self, node):
        return type(node) not in [ast.Load, ast.Store]

    def add_node(self, node):
        node_value = node.value

        if isinstance(node_value, ast.Name):
            label_i = node_value.id
        elif isinstance(node_value, ast.Constant):
            label_i = node_value.n
        elif isinstance(node_value, ast.arg):
            label_i = node_value.arg
        elif isinstance(node_value, ast.BinOp):
            label_i = type(node_value.op).__name__
        elif isinstance(node_value, ast.UnaryOp):
            label_i = type(node_value.op).__name__
        else:
            label_i = type(node_value).__name__

        self.G.add_node(node, label=label_i, style='filled', fillcolor=f'#{hex(31337*hash(type(node.value))%(256**3))[2:]}')

    def visit(self, node):
        for i in ast.iter_child_nodes(node.value):
            if not self.go_further(i):
                continue

            node_i = self.AntiSingleton(i)

            self.add_node(node_i)

            self.G.add_edge(node, node_i)
            self.visit(node_i)

In [26]:
nx.drawing.nx_pydot.to_pydot(ASTBuilder(main_ast).G).write_png('artifacts/tree.png')